In [626]:
# para tratar datasets desbalanceados
!pip install imbalanced-learn

In [627]:
# para utilizar no lemmatizer com um dicionário em pt-BR
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [628]:
import pandas as pd
import joblib
import nltk
import re
import string
import unicodedata
import spacy
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings

In [629]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [630]:
# NTLK
nltk.download('stopwords')
nltk.download('punkt')

# Spacy
nlp_spacy_ptbr = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [631]:
df = pd.read_csv('/content/processado_base_full.csv', sep=';')
df.head()

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
0,Escola Pública,ALUNO-1,11.0,2.0,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Escola Pública,ALUNO-3,12.0,2.0,3H,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Escola Pública,ALUNO-4,10.0,2.0,1D,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Escola Pública,ALUNO-5,10.0,1.0,2M,Não,8.077085,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [632]:
df['PONTO_VIRADA'].value_counts()

PONTO_VIRADA
Não    1932
Sim     315
Name: count, dtype: int64

In [633]:
df.query('PONTO_VIRADA == "Sim"').head(10)

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.750000,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Escola Pública,ALUNO-11,8.0,0.0,0A,Sim,9.712504,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.00002,10.0,8.125,10.000000,10.000000,10.000010,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Rede Decisão/União,ALUNO-28,13.0,2.0,3J,Sim,9.354726,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.58335,9.6,6.875,9.000000,10.000000,9.944453,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Rede Decisão/União,ALUNO-55,17.0,3.0,6C,Sim,6.878125,C,Ágata,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,0.00000,9.0,7.500,6.333333,8.281250,8.666667,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Rede Decisão/União,ALUNO-75,11.0,1.0,3L,Sim,9.687504,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,10.00002,10.0,10.000,9.000000,9.375000,9.750010,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Escola Pública,ALUNO-81,15.0,1.0,4L,Sim,7.386391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,5.41667,8.8,7.500,6.000000,8.125000,9.111120,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Rede Decisão/União,ALUNO-83,14.0,4.0,4B,Sim,8.155003,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,9.16668,9.4,5.625,5.333333,8.958333,9.166677,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Rede Decisão/União,ALUNO-87,14.0,4.0,4L,Sim,8.188613,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.2,5.625,6.666667,8.750000,8.722227,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Rede Decisão/União,ALUNO-88,11.0,2.0,2D,Sim,9.040420,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,10.00002,9.4,7.500,9.500000,5.937500,9.583340,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Escola Pública,ALUNO-96,18.0,1.0,7A,Sim,6.432501,C,Ágata,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,9.16668,6.1,6.875,3.000000,7.500000,8.791665,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [634]:
df_ml = df[['PONTO_VIRADA', 'DESTAQUE_IEG',	'DESTAQUE_IDA',	'DESTAQUE_IPV',	'INDE', 'IAA',	'IEG',	'IPS',	'IDA', 	'IPP',	'IPV',	'IAN']]
df_ml.head()

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,INDE,IAA,IEG,IPS,IDA,IPP,IPV,IAN
0,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.883752,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0
1,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.856390,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0
2,Não,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,5.076252,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0
3,Não,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,8.077085,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0
4,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.381391,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0


In [635]:
modelo = joblib.load('/content/model.pkl')
vect = joblib.load("/content/vect.pkl")

In [636]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

def lemmatize(text):
    doc = nlp_spacy_ptbr(text)
    return " ".join([token.lemma_ for token in doc])

def predict_sentiment(texto):
    texto_teste = tokenizer(texto)
    texto_lematizado = lemmatize(texto_teste)
    text_vect = vect.transform([texto_lematizado])
    return modelo.predict(text_vect)[0]

In [637]:
clone = df_ml.copy()

In [638]:
clone['PONTO_VIRADA'].value_counts(dropna=False)

PONTO_VIRADA
Não    1932
Sim     315
NaN      26
Name: count, dtype: int64

In [639]:
clone['PONTO_VIRADA'][clone['PONTO_VIRADA'].isna()].head()

27     NaN
86     NaN
94     NaN
118    NaN
161    NaN
Name: PONTO_VIRADA, dtype: object

In [640]:
clone.iloc[27].to_frame().T

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,INDE,IAA,IEG,IPS,IDA,IPP,IPV,IAN
27,NaN,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,8.696668,9.16668,10.0,7.5,8.2,0.0,0.0,10.0


In [641]:
clone = clone.dropna(subset=['PONTO_VIRADA'])

In [642]:
clone.query('PONTO_VIRADA == "Sim"').head(10)

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,INDE,IAA,IEG,IPS,IDA,IPP,IPV,IAN
4,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.381391,8.33334,9.9,4.375,7.333333,8.750000,8.944450,10.0
6,Sim,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.712504,9.00002,10.0,8.125,10.000000,10.000000,10.000010,10.0
12,Sim,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.354726,9.58335,9.6,6.875,9.000000,10.000000,9.944453,10.0
28,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,6.878125,0.00000,9.0,7.500,6.333333,8.281250,8.666667,5.0
36,Sim,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.687504,10.00002,10.0,10.000,9.000000,9.375000,9.750010,10.0
40,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,7.386391,5.41667,8.8,7.500,6.000000,8.125000,9.111120,5.0
42,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.155003,9.16668,9.4,5.625,5.333333,8.958333,9.166677,10.0
46,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.188613,8.33334,9.2,5.625,6.666667,8.750000,8.722227,10.0
47,Sim,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.040420,10.00002,9.4,7.500,9.500000,5.937500,9.583340,10.0
53,Sim,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,6.432501,9.16668,6.1,6.875,3.000000,7.500000,8.791665,5.0


In [643]:
clone['PONTO_VIRADA'].value_counts(dropna=False)

PONTO_VIRADA
Não    1932
Sim     315
Name: count, dtype: int64

In [644]:
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()
# scaler = StandardScaler() # NOTA: desativado o scaler... ele impactava a previsão final de forma negativa

In [645]:
def processar_sentimento_nlp_df(df):
  df['DESTAQUE_IEG_NLP'] = df['DESTAQUE_IEG'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
  df['DESTAQUE_IDA_NLP'] = df['DESTAQUE_IDA'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
  df['DESTAQUE_IPV_NLP'] = df['DESTAQUE_IPV'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')

  return df

def aplicar_onehot_encoder(df):
  # colunas que vão receber o OneHotEncoder
  cols_to_encode = ['DESTAQUE_IDA_NLP', 'DESTAQUE_IEG_NLP', 'DESTAQUE_IPV_NLP']
  df[cols_to_encode] = df[cols_to_encode].astype(str)

  # dropa as colunas originais que o NLP processou
  df.drop(['DESTAQUE_IDA', 'DESTAQUE_IEG', 'DESTAQUE_IPV'], axis=1, inplace=True)

  # aplica o OneHotEncoder
  codificado = onehot_encoder.fit_transform(df[cols_to_encode])
  colunas_codificadas = onehot_encoder.get_feature_names_out(cols_to_encode)
  df_codificado = pd.DataFrame(codificado, columns=colunas_codificadas)

  # pode ser que um registro não possua todas as colunas finais do OneHotEncoder (p.ex. quando para a coluna IPV não é classificada como neutra),
  # entretanto, o NLP ainda pode eventualmente classificar tal feature como neutra, o que quebraria o modelo em tal caso
  # o código a seguir, garante que todas as linhas do df possuam as variantes geradas pelo NLP
  colunas_necessarias = [
      'DESTAQUE_IDA_NLP_NA', 'DESTAQUE_IDA_NLP_negativo', 'DESTAQUE_IDA_NLP_neutro', 'DESTAQUE_IDA_NLP_positivo',
      'DESTAQUE_IEG_NLP_NA', 'DESTAQUE_IEG_NLP_negativo', 'DESTAQUE_IEG_NLP_neutro', 'DESTAQUE_IEG_NLP_positivo',
      'DESTAQUE_IPV_NLP_NA', 'DESTAQUE_IPV_NLP_negativo', 'DESTAQUE_IPV_NLP_neutro', 'DESTAQUE_IPV_NLP_positivo'
  ]

  for col in colunas_necessarias:
        if col not in df_codificado.columns:
            df_codificado[col] = 0

  # reseta o index para o concat
  df_codificado = df_codificado.reset_index(drop=True)
  df = df.reset_index(drop=True)

  # concat final das colunas (OneHotEncoder + colunas originais - colunas originais que deveriam ser encodadas)
  df_final = pd.concat([df_codificado, df.drop(columns=cols_to_encode)], axis=1)

  return df_final

def aplicar_scaler(df):
  clone = df.copy()
  # clone_scaled = scaler.fit_transform(df)
  clone_scaled = clone # NOTA: desativado o scaler... ele impactava a previsão final de forma negativa

  return clone_scaled

def preparar_dados(df):
  df = processar_sentimento_nlp_df(df)
  df = aplicar_onehot_encoder(df)
  df = df[sorted(df.columns)] # para garantir que o input sempre será enviado para o scaler na ordem correta

  return df

In [646]:
X = aplicar_scaler(preparar_dados(clone.drop('PONTO_VIRADA', axis=1)))
y = pd.Series(label_encoder.fit_transform(clone['PONTO_VIRADA']))

In [647]:
label_encoder.classes_

array(['Não', 'Sim'], dtype=object)

In [648]:
y.value_counts()

0    1932
1     315
Name: count, dtype: int64

In [649]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=777)

In [650]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [651]:
y_train_resampled.value_counts()

0    1352
1    1352
Name: count, dtype: int64

In [652]:
def melhores_hiperparamentros_xgb(X_train, y_train):
    param_grid = {
      'learning_rate':[0.01, 0.05, 0.2],
      'n_estimators':[50, 100, 200, 500],
      'max_depth': [1, 2, 5, 8]
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)

    grid = GridSearchCV(
      GradientBoostingClassifier(),
      param_grid = param_grid,
      scoring = 'f1',
      cv = cv,
      n_jobs= -1
    )

    # seleciona os melhores hiperparâmetros com base nos dados de treino
    grid.fit(X_train, y_train)
    xgb_params = grid.best_params_

    print("Melhores hiperparâmetros para XGB:", xgb_params)

    return xgb_params

In [653]:
%%time
# NOTA: desativado pois é muito pesado para rodar
# params = melhores_hiperparamentros_xgb(X_train, y_train)
# params

# Melhores hiperparâmetros para XGB: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
# {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [654]:
kfold = KFold(n_splits=5, shuffle=True)
xgb = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=300)
xgb.fit(X_train, y_train)
xgb_result = cross_val_score(xgb, X_train_resampled, y_train_resampled, cv=kfold)

In [655]:
print(f'Média das pontuações de validação cruzada: {xgb_result.mean()}')
print(f'Desvio padrão das pontuações: {xgb_result.std()}')

Média das pontuações de validação cruzada: 0.9940829739166155
Desvio padrão das pontuações: 0.0007383553410132701


In [656]:
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       580
           1       0.99      0.95      0.97        95

    accuracy                           0.99       675
   macro avg       0.99      0.97      0.98       675
weighted avg       0.99      0.99      0.99       675



In [657]:
xgb_acuracia = accuracy_score(y_true=y_test, y_pred=y_pred) * 100
xgb_acuracia

99.11111111111111

In [658]:
def xgb_predict(aluno):
    aluno = preparar_dados(aluno.drop('PONTO_VIRADA', axis=1))
    aluno_scaled = aplicar_scaler(aluno)

    display('aluno', aluno)
    display('aluno scaled', aluno_scaled)

    return xgb.predict(aluno_scaled)

In [659]:
df_ml.iloc[0].to_frame().T

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,INDE,IAA,IEG,IPS,IDA,IPP,IPV,IAN
0,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.883752,8.50002,8.7,7.5,7.0,5.9375,7.75,10.0


In [660]:
df_ml = df[['PONTO_VIRADA', 'DESTAQUE_IEG', 'DESTAQUE_IDA', 'DESTAQUE_IPV', 'INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']].query('DESTAQUE_IEG.notna() and DESTAQUE_IDA.notna() and DESTAQUE_IPV.notna()')
aluno = df_ml.iloc[0].to_frame().T
aluno

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,INDE,IAA,IEG,IPS,IDA,IPP,IPV,IAN
0,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.883752,8.50002,8.7,7.5,7.0,5.9375,7.75,10.0


In [661]:
xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,0,0,1.0,0,0,0,1.0,0,0,0,1.0,8.50002,10.0,7.0,8.7,7.883752,5.9375,7.5,7.75


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,0,0,1.0,0,0,0,1.0,0,0,0,1.0,8.50002,10.0,7.0,8.7,7.883752,5.9375,7.5,7.75


array([0])

In [662]:
aluno = pd.DataFrame({
  'PONTO_VIRADA': 'Sim',
  'DESTAQUE_IEG': 'Ótimo aluno! Muito bom, continue assim! Você está indo muito bem',
  'DESTAQUE_IDA': 'Horrivel, muito ruim, tem que melhorar muito ainda.',
  'DESTAQUE_IPV': 'Bem mediano, mas poderia melhorar as notas e se esforçar mais do que fez',
  'INDE': 10,
  'IAA': 10,
  'IEG': 10,
  'IPS': 10,
  'IDA': 10,
  'IPP': 10,
  'IPV': 10,
  'IAN': 10
}, index=[0])

xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,0,0,1.0,0,0,0,1.0,10,10,10,10,10,10,10,10


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,0,0,1.0,0,0,0,1.0,10,10,10,10,10,10,10,10


array([1])

In [663]:
clone.iloc[4].to_frame().T['DESTAQUE_IPV'].values

array(['Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.'],
      dtype=object)

In [664]:
aluno = pd.DataFrame({
  'PONTO_VIRADA': 'Sim',
  'DESTAQUE_IEG': 'Seu destaque em 2020: A sua boa entrega das lições de casa.',
  'DESTAQUE_IDA': 'Ponto a melhorar em 2021: Empenhar-se mais nas aulas e avaliações.',
  'DESTAQUE_IPV': 'Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.',
  'INDE': 8.381391,
  'IAA': 8.33334,
  'IEG': 9.9,
  'IPS': 4.375,
  'IDA': 7.333333,
  'IPP': 8.750000,
  'IPV': 8.944450,
  'IAN': 10.0
}, index=[0])

xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,0,0,1.0,0,0,0,1.0,0,0,0,1.0,8.33334,10.0,7.333333,9.9,8.381391,8.75,4.375,8.94445


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,0,0,1.0,0,0,0,1.0,0,0,0,1.0,8.33334,10.0,7.333333,9.9,8.381391,8.75,4.375,8.94445


array([1])

In [665]:
aluno = pd.DataFrame({
  'PONTO_VIRADA': 'Não',
  'DESTAQUE_IEG': 'Péssimo aluno, muito ruim, horrivel',
  'DESTAQUE_IDA': 'Péssimo aluno, muito ruim, horrivel',
  'DESTAQUE_IPV': 'Péssimo aluno, muito ruim, horrivel',
  'INDE': 0,
  'IAA': 0,
  'IEG': 0,
  'IPS': 0,
  'IDA': 0,
  'IPP': 0,
  'IPV': 0,
  'IAN': 0
}, index=[0])

xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,1.0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,1.0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0


array([0])

# Exportando o modelo

In [666]:
xgb_file = 'xgb.pkl'
labelencoder_file = 'labelencoder.pkl'
onehotencoder_file = 'onehotencoder.pkl'

In [667]:
joblib.dump(xgb, xgb_file)
joblib.dump(label_encoder, labelencoder_file)
joblib.dump(onehot_encoder, onehotencoder_file)

['onehotencoder.pkl']

# Importando o modelo

In [669]:
xgb = joblib.load(xgb_file)
label_encoder = joblib.load(labelencoder_file)
onehot_encoder = joblib.load(onehotencoder_file)

In [670]:
aluno = pd.DataFrame({
  'PONTO_VIRADA': 'Não',
  'DESTAQUE_IEG': 'Péssimo aluno, muito ruim, horrivel',
  'DESTAQUE_IDA': 'Péssimo aluno, muito ruim, horrivel',
  'DESTAQUE_IPV': 'Péssimo aluno, muito ruim, horrivel',
  'INDE': 0,
  'IAA': 0,
  'IEG': 0,
  'IPS': 0,
  'IDA': 0,
  'IPP': 0,
  'IPV': 0,
  'IAN': 0
}, index=[0])

xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,1.0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,1.0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0


array([0])

In [671]:
aluno = pd.DataFrame({
  'PONTO_VIRADA': 'Sim',
  'DESTAQUE_IEG': 'Ótimo aluno! Muito bom, continue assim! Você está indo muito bem',
  'DESTAQUE_IDA': 'Horrivel, muito ruim, tem que melhorar muito ainda.',
  'DESTAQUE_IPV': 'Bem mediano, mas poderia melhorar as notas e se esforçar mais do que fez',
  'INDE': 10,
  'IAA': 10,
  'IEG': 10,
  'IPS': 10,
  'IDA': 10,
  'IPP': 10,
  'IPV': 10,
  'IAN': 10
}, index=[0])

xgb_predict(aluno)

'aluno'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,0,0,1.0,0,0,0,1.0,10,10,10,10,10,10,10,10


'aluno scaled'

,DESTAQUE_IDA_NLP_NA,DESTAQUE_IDA_NLP_negativo,DESTAQUE_IDA_NLP_neutro,DESTAQUE_IDA_NLP_positivo,DESTAQUE_IEG_NLP_NA,DESTAQUE_IEG_NLP_negativo,DESTAQUE_IEG_NLP_neutro,DESTAQUE_IEG_NLP_positivo,DESTAQUE_IPV_NLP_NA,DESTAQUE_IPV_NLP_negativo,DESTAQUE_IPV_NLP_neutro,DESTAQUE_IPV_NLP_positivo,IAA,IAN,IDA,IEG,INDE,IPP,IPS,IPV
0,0,1.0,0,0,0,0,0,1.0,0,0,0,1.0,10,10,10,10,10,10,10,10


array([1])